# Change detection using M3C2 algorithm
With the help of the change detection algorithm M3C2 we can remove non-static objects from the point cloud. Resulting in a point cloud with only static objects, that we name obstacles. The provided example tile in the folder `../datasets/pointclouds/sidewalk/m3c2/` is generated from the M3C2 plugin inside CloudCompare. This notebook presents how the M3C2 algorithm can be run using the CloudComPy package.

In [ ]:
import os
import sys
import math
os.environ["_CCTRACE_"]="ON"

import cloudComPy as cc
import numpy as np
import time

In [ ]:
cc.initCC()  # to do once before using plugins or dealing with numpy

Load two datasets that cover the same scene at two different points in time.

In [ ]:
# Load point clouds 
cloud = cc.loadPointCloud('../datasets/pointclouds/sidewalk/run1/processed_2386_9702.txt')
cloud1 = cc.loadPointCloud('../datasets/pointclouds/sidewalk/run2/processed_2386_9702.txt')

The distance analysis is executed on a number of points of interest called *core points*. This could be e.g. the entire reference point cloud, a downsampled version of it, an equistant grid etc. In the m3c2_params configuration file, we choose the entire point cloud:

In [ ]:
paramFilename = "m3c2_params.txt" # Validate MaxThreadCount -> multiprocessing.cpu_count()

Next, we instantiate the algorithm class and run the distance calculation:

In [ ]:
start = time.time()

if cc.isPluginM3C2():
    import cloudComPy.M3C2
    
    cloud2 = cc.M3C2.computeM3C2([cloud,cloud1], paramFilename)

    if cloud2 is None:
        raise RuntimeError
    if cloud2.getNumberOfScalarFields() < 3:
        raise RuntimeError
    dic = cloud2.getScalarFieldDic()
    # The calculated distances
    sf = cloud2.getScalarField(dic['M3C2 distance'])
    if sf is None:
        raise RuntimeError

    cc.SaveEntities([cloud, cloud1, cloud2], "M3C2.bin")

    end = time.time()
    print(end - start)